In [39]:
import torch
from torch import nn, optim
from torch import functional as F
import torchvision
from torchvision import transforms
import numpy as np

In [12]:
# буду использовать MNIST в качестве датасета
transform = transforms.Compose( [transforms.ToTensor(), transforms.Normalize( (0.13,), (0.31,) )] )

train_data = torchvision.datasets.MNIST(root='./data', 
                                        train=True, 
                                        download=False, 
                                        transform=transform)

test_data = torchvision.datasets.MNIST(root='./data', 
                                       train=False, 
                                       download=False, 
                                       transform=transform)

train_loader = torch.utils.data.DataLoader(dataset=train_data, batch_size=4, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_data, batch_size=4, shuffle=False)

In [43]:
class ResBlock(nn.Module):
    def __init__(self, inchannels, outchannels, stride=1):
        super(ResBlock, self).__init__()
        
        self.stride = stride
        self.conv1 = nn.Conv2d(in_channels=inchannels, 
                               out_channels=outchannels, kernel_size=3, stride=self.stride)
        self.conv2 = nn.Conv2d(in_channels=outchannels, 
                               out_channels=outchannels, kernel_size=3, stride=self.stride)
        
        self.bn = nn.BatchNorm2d(num_features=outchannels)
        self.relu = nn.ReLU(inplace=True)
        
    def forward(self, x):
        identity = x
        
        out = self.conv1(x)
        out = self.bn(self.relu(out))
        
        out = self.conv2(out)
        out = self.bn(out) # думаю, можно без этого 
        out += identity # тут надо "объединить" вход и аппроксмирующую ф-ию
        
        out = self.relu(out)
        
        return nn.Sequential( out )
        
        

In [44]:
class ResNet(nn.Module):
    
    def __init__(self, inchannels, n_layers, layer_comb, n_classes=10):
        """
        layer_comb : лист, например, для 18слойной сети - это [2, 2, 2, 2]
        """
        super(ResNet, self).__init__()

        self.inchannels = inchannels
        
        # начальные слои, еще до ухода в глубину
        self.conv1 = nn.Conv2d(in_channels=inchannels, 
                               out_channels=64, kernel_size=7, stride=2, padding=3)
        self.bn = nn.BatchNorm2d(64) 
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        self.conv2_x = self.make_layer(outchannels=64,  n_resblocks=layer_comb[0])
        self.conv3_x = self.make_layer(outchannels=128, n_resblocks=layer_comb[1], stride=2)
        self.conv4_x = self.make_layer(outchannels=256, n_resblocks=layer_comb[2], stride=2)
        self.conv5_x = self.make_layer(outchannels=512, n_resblocks=layer_comb[3], stride=2)
        
        self.avgpool = nn.AvgPool2d(kernel_size=(1, 1))
        self.fc = nn.Linear(1000, n_classes)
        

    ###----- надо доработать! ----------------------------------------
    def make_layer(self, outchannels, n_resblocks, stride=1):
        
        layers = []
        
        for _ in range(n_resblocks):
            layers.append(ResBlock(self.inchannels, outchannels, stride))
            
        return nn.Sequential(*layers)
            
    def forward(self, x):
        x = self.conv1(x)
        x = self.bn(self.relu(x))
        x = self.maxpool(x)
        
        x = self.conv2_x(x) # TODO
        x = self.conv3_x(x) # TODO
        x = self.conv4_x(x) # TODO
        x = self.conv5_x(x) # TODO
        
        x = self.avgpool(x) 
        x = nn.Flatten(x) # flat it before fully connected
        x = self.fc(x) 
        x = F.softmax(x) # на конце сглаживаем максимум как написано в статье
        
        return x
        
        
        
        
        

In [45]:
model = ResBlock(1, 64)

In [46]:
optimizer = optim.Adadelta(model.parameters(), lr=0.01)
for epoch in range(6):
    
    for b_idx, (data, target) in enumerate(train_loader):
        # zero initialization of gradients
        optimizer.zero_grad()
    
        # feed to model , forward pass
        out = model(data)
        loss = F.nll_loss(out, target)
    
        # backprop
        loss.backward()
        optimizer.step()
    
        if b_idx % 1000 == 0 and i != 0:
            print("epoch is {} and loss is {}".format(epoch, loss.item()))

RuntimeError: The size of tensor a (24) must match the size of tensor b (28) at non-singleton dimension 3

In [37]:
train_loader.dataset.data.shape

torch.Size([60000, 28, 28])